# Benchmark

Actually not so much a benchmark, but a visualization on the number of possible paths as a function of the number of number of steps, and fsms paramters.

In [24]:
from pathlib import Path
import json

from solarmed_optimization.visualization import paths_per_initial_state_per_system_viz

%load_ext autoreload
%autoreload 2

data_path: Path = Path("../results")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
with open(data_path / "metadata.json", mode="r") as f:
    metadata = json.load(f)

# metadata


In [ ]:
figs = paths_per_initial_state_per_system_viz(metadata)

[fig.show() for fig in figs]


[None, None]

In [ ]:
systems: list[str] = ["SFTS", "MED"]
y_label: str = "n of paths"
states_per_system = {
    system: list(metadata[system][0]["n_paths_per_initial_state"].keys())
    for system in systems
}
initial_states = {
    system: states_per_system[system]
    for system in systems
}
initial_state = "IDLE"

results = [result for result in  metadata[systems[0]]]

# Prepare data
data = {
    'x': [f'N={d["n_horizon"]}' for d in results],
    # 'color': alternatives,
    'facet_col': systems,
}



In [19]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from solarmed_optimization.visualization import update_n_paths_plot
from IPython.display import display
from ipywidgets import interact, fixed, Output, HBox, widgets
import pandas as pd

def get_nested_attr(d, attr):
        keys = attr.split('.')
        for key in keys:
            d = d.get(key, None)
        return d

# To modify the gaps between subplots, you can modify the domain of their xaxis
def domains_calculator(gap, n_plot):
    """
    little helper to calculate the range of subplots domains, by DIDIER Sébastien
    https://community.plotly.com/t/control-distance-between-stacked-bars/75303/4
    """
    plot_width = (1 - (n_plot-1) * gap) / n_plot
    return [
        [i * (plot_width + gap), i * (plot_width + gap) + plot_width]
        for i in range(n_plot)
    ]
    
def generate_plot(metadata, system, initial_states) -> go.Figure | None:
    """ Generate a plot for a given y_var and y_label
    
    Based on DIDIER Sébastien suggestion in [plotly forum](https://community.plotly.com/t/control-distance-between-stacked-bars/75303/4)

    Args:
        y_var (str): Name of the variable in `results`
        y_label (str): Label for the y axis

    Returns:
        go.Figure | None: Plotly figure object
    """
    # systems = list(metadata.keys())

    n_plots = len(initial_states[system])

    y = []
    x = []
    facet_col = []
    for initial_state in initial_states[system]:
        for d in metadata[system]:
            # y.append( 
            #     get_nested_attr(d, f"n_paths_per_initial_state.{initial_state}")
            # )
            y.append(d["n_paths_per_initial_state"][initial_state]),
            x.append(f'N={d["n_horizon"]}'),
            facet_col.append(initial_state)
        
    # Flatten the metadata
    data = [d for d in metadata[system] for system in systems]

    # Prepare data
    data = dict(
        x=x,
        y = y,
        facet_col=facet_col,
        # 'color': alternatives,
    )
    # print(data)
    # data = pd.DataFrame()
    # print(data)

    fig = px.bar(data, x="x", y="y", facet_col="facet_col") # color="color", facet_col="facet_col", pattern_shape="color"

    # little customisations
    fig.for_each_annotation(lambda a: a.update(text=''))# remove the facet titles
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)', 
        barmode='group', 
        legend=dict(title='Color legend', x=0.8, y=1.3),
        # margin=dict(l=0, r=0, t=100, b=20),
        width = n_plots * 300, #  if width is None else width,
        title=f"Number of paths per initial state for <b>{system}</b>"
    )
    fig.update_yaxes(title_text=y_label, ticks="outside", linecolor="black", row=1, col=1)
    
    domains = domains_calculator(gap=0.05, n_plot=n_plots)

    [fig.update_xaxes(title_text=f"<b>{intial_state}</b>", domain=domains[idx], row=1, col=idx+1) for idx, intial_state in enumerate(initial_states[system])]
    [fig.update_yaxes(showgrid=True, gridcolor='lightgray', row=1, col=idx+1) for idx in range(len(initial_states[system]))]

    return fig


# fig = generate_plot(metadata, system="SFTS", initial_states=initial_states) 

figs = [generate_plot(metadata, system=system, initial_states=initial_states) for system in systems]

# # Create the figure and Output widget
# output = Output()

# kwargs = {}
# widget_list = []
# for system in systems:
#     kwargs.update({f"initial_state_{system}": [state for state in initial_states[system]] })
#     widget_list.append(
#         widgets.Dropdown( description = f"Initial state {system}", options=initial_states[system])
#     )
    

# def update_figure(**kwargs):
#     with output:  # Use the output widget to control display
#         with fig.batch_update():
#             idx = 0
#             for initial_state, system in zip(kwargs.values(), systems):
#                 fig.data[idx].y = [d['n_paths_per_initial_state'][initial_state] for d in metadata[system]]
#                 idx += 1
# # Display the Output widget itself (for the interactive part)
# interact_manual = widgets.interactive(update_figure, **kwargs)
# with output:
#     display(HBox(widget_list))  # Arrange widgets horizontally
#     display(fig)

# display(output)

[fig.show() for fig in figs]


[None, None]